In [1]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import pandas as pd
import numpy as np
import gzip
import json
import math
from datetime import datetime
from IPython.display import display
%matplotlib inline

In [2]:
from helper.manage_orderbooks_v2 import extract_orderbooks_for_one_currencypair, load_orderbook_snapshot
from helper.orderbook_container import OrderbookContainer
from helper.orderbook_trader import OrderbookTradingSimulator

In [3]:
filename = '3000orderbooks'
data = load_orderbook_snapshot(filename, last_line=200)

100%|██████████| 200/200 [00:05<00:00, 39.88it/s]

Loaded 200 orderbooks from file '3000orderbooks'.


In [4]:
orderbook = data[0]
orderbook.plot(range_factor=1.02, outfile="sample.pdf")
print(orderbook)
orderbook2 = data[1]
print(orderbook2)

print("\nCompare two orderbooks:")
diff = orderbook.compare_with(other=orderbook2)

display(diff.head())
print(diff)

Successfully saved 'sample.pdf'
OrderbookContainer from 2016-11-08T10:00
  499 bids (best: 705.0)
  407 asks (best: 705.45)
  kind: 'orderbook'
OrderbookContainer from 2016-11-08T10:01
  499 bids (best: 703.8)
  406 asks (best: 705.45)
  kind: 'orderbook'

Compare two orderbooks:


,Amount,Volume,VolumeAcc,norm_Price,Type
685.11,0.473604,324.471015,108526.174016,0.971477,bid
678.34,0.511551,347.005553,142564.324900,0.961877,bid
674.59,0.546871,368.913775,150950.334759,0.956560,bid
705.45,-1.876972,2237.815171,2237.815171,1.000319,ask
706.89,0.105160,74.336899,2805.184977,1.002361,ask
707.18,6.456261,5649.346841,8454.531818,1.002772,ask


OrderbookContainer from 2016-11-08T10:01
  12 bids (best: 674.59)
  15 asks (best: 705.45)
  kind: 'diff'


In [5]:
vol = 40
print("Current MarketOrderPrice for {} shares: {}".format(vol, orderbook.get_current_price(vol)))

Current MarketOrderPrice for 40 shares: 28358.1256107


In [6]:
display(orderbook.head(2))
display(orderbook.tail(2))

,Amount,Type,Volume,VolumeAcc,norm_Price
703.700000,0.001250,bid,0.879625,112.433157,0.997838
705.000000,0.158232,bid,111.553532,111.553532,0.999681
705.224976,NaN,center,NaN,NaN,NaN
705.450000,3.172181,ask,2237.815171,2237.815171,1.000319
706.170000,0.052838,ask,37.312497,2275.127669,1.001340


,Amount,Type,Volume,VolumeAcc,norm_Price
614.000000,1.838163,bid,1128.632254,326770.989935,0.870644
614.320000,0.034763,bid,21.355643,325642.357681,0.871098
705.224976,NaN,center,NaN,NaN,NaN
810.000000,0.286798,ask,232.306137,292461.937892,1.148570
811.000000,0.000100,ask,0.081100,292462.018992,1.149988


In [7]:
P = 10
vol=20
T = 12
ots = OrderbookTradingSimulator(volume=vol, tradingperiods=T, decisionfrequency=P)

for e in tqdm(range(T)):
    res = ots.trade(orderbooks=data[P*e:P*(e+1)], limit=707., verbose=True)
#    vol = ots.history.volume_left.values[-1]
# res = ots.trade_timespan(orderbooks=data[2:4], volume=40, limit=705.5, timespan=2)

display(ots.history)
display(res)

  8%|▊         | 1/12 [00:00<00:04,  2.38it/s]

t=10: Traded 15.2370/20.0000 shares for -10759.8754664, 4.7630 shares left


 33%|███▎      | 4/12 [00:01<00:03,  2.49it/s]

t=40: Traded 4.6358/4.7630 shares for -3271.51483807, 0.1273 shares left


 75%|███████▌  | 9/12 [00:02<00:00,  3.71it/s]

No shares left at t=6 (self.t=47), Done!
t=47: Traded 0.1273/0.1273 shares for -89.886612311, 0.0000 shares left


100%|██████████| 12/12 [00:02<00:00,  5.70it/s]


,ASK,BID,CENTER,LIMIT,SPREAD,T,VOLUME,avg,cashflow,cost,cost_avg,high,low,volume_traded
2016-11-08T10:00,705.45,614.0,658.667256,707.0,91.45,10,20.000000,706.16874,-10759.875466,10.951410,0.71874,706.89,705.000000,15.236975
2016-11-08T10:10,707.15,614.0,659.478928,707.0,93.15,10,4.763025,0.00000,0.000000,0.000000,0.00000,0.00,inf,0.000000
2016-11-08T10:20,707.10,614.0,659.455064,707.0,93.10,10,4.763025,0.00000,0.000000,0.000000,0.00000,0.00,inf,0.000000
2016-11-08T10:30,707.10,614.0,659.455064,707.0,93.10,10,4.763025,705.71401,-3271.514838,1.223900,0.26401,707.00,703.800000,4.635752
2016-11-08T10:40,706.19,614.0,659.020650,707.0,92.19,10,0.127273,706.24833,-89.886612,0.101606,0.79833,706.83,703.630000,0.127273


OrderbookContainer from 2016-11-08T11:51
  512 bids (best: 614.32)
  406 asks (best: 707.82)
  kind: 'orderbook'